In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

2021-08-16 16:02:27.236580: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Cargando datos

In [2]:
path_to_file = '../data/txts/chamame_dataset.txt'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 44425 characters


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

69 unique characters


# Procesamiento de texto

In [6]:
# Split into tokens
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(44425,), dtype=int64, numpy=array([59, 32, 45, ...,  3,  1,  1])>

In [7]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(44425,), dtype=string, numpy=array([b'\xc3\x91', b'a', b'n', ..., b' ', b'\n', b'\n'], dtype=object)>

In [8]:
# Join characters back into strings
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Creando pares entrenamiento-objetivo

In [9]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [10]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Ñ
a
n
g
a
p
i
r
í
 


In [11]:
# Batch definition
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xc3\x91' b'a' b'n' b'g' b'a' b'p' b'i' b'r' b'\xc3\xad' b' ' b's' b'i'
 b'l' b'v' b'e' b's' b't' b'r' b'e' b' ' b'l' b'u' b'z' b' ' b'd' b'e'
 b'l' b' ' b'a' b'r' b'e' b'n' b'a' b'l' b' ' b'\n' b't' b'i' b'b' b'i'
 b'o' b' ' b'r' b'u' b'b' b'o' b'r' b' ' b'd' b'e' b'l' b' ' b'v' b'e'
 b'r' b'd' b'e' b' ' b'g' b'r' b'i' b's' b' ' b'd' b'e' b' ' b'l' b'a'
 b' ' b'n' b'i' b'\xc3\xb1' b'e' b'z' b'.' b' ' b'\n' b'\n' b'\n' b'M'
 b'i' b' ' b'r' b'e' b's' b'p' b'l' b'a' b'n' b'd' b'o' b'r' b' ' b'm'
 b'i' b' ' b'a' b't' b'a' b'r' b'd'], shape=(101,), dtype=string)


In [12]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\xc3\x91angapir\xc3\xad silvestre luz del arenal \ntibio rubor del verde gris de la ni\xc3\xb1ez. \n\n\nMi resplandor mi atard'
b'ecer mi tarag\xc3\xbci \ny el cielo aqu\xc3\xad de tu mirar \xc3\xb1angapir\xc3\xad. \n\n\nAnduve mirando el pueblo busc\xc3\xa1ndome \ny flu'
b'yes como mi sangre de chamam\xc3\xa9 \nenciende tu pie desnudo sobre mi piel \nun r\xc3\xado de primavera ternura y m'
b'iel \na orillas de tu sonrisa yo soy feliz \nno quiero partir de nuevo \xc3\xb1angapir\xc3\xad. \n\n\nLa luna se ardi\xc3\xb3 e'
b'n el r\xc3\xado \xc3\xb1angapir\xc3\xad \ny vuela a lo m\xc3\xa1s profundo de tu vivir \nall\xc3\xad donde es todo aroma va mi canci\xc3\xb3n \na '


In [13]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [14]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'\xc3\x91angapir\xc3\xad silvestre luz del arenal \ntibio rubor del verde gris de la ni\xc3\xb1ez. \n\n\nMi resplandor mi atar'
Target: b'angapir\xc3\xad silvestre luz del arenal \ntibio rubor del verde gris de la ni\xc3\xb1ez. \n\n\nMi resplandor mi atard'


2021-08-16 16:02:41.778217: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-16 16:02:41.797859: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


# Creando baches de entrenamiento

In [15]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

# Modelo 

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [17]:
# Check the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


2021-08-16 16:02:44.717998: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-16 16:02:45.213368: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-16 16:02:45.370622: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


(32, 100, 70) # (batch_size, sequence_length, vocab_size)


2021-08-16 16:02:45.894941: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


In [18]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17920     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  71750     
Total params: 4,027,974
Trainable params: 4,027,974
Non-trainable params: 0
_________________________________________________________________


## Prueba del modelo
To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [20]:
sampled_indices

array([40, 63, 50,  0,  8,  3,  3, 27, 30, 45, 26,  8, 48, 24, 35, 17, 56,
       52,  3, 55, 65, 13, 42, 16, 64, 54, 21, 50, 15, 13, 63, 29, 33,  2,
       51,  9, 30, 34, 45, 55, 66, 41, 25, 68, 28,  1, 66,  9, 66, 25, 59,
       33, 36, 19, 46, 37, 32, 36, 27, 69, 17,  3, 48, 61, 17, 41, 44, 47,
       56, 52,  4,  0, 25, 23, 13, 28, 31, 52, 56, 11, 54, 23, 69, 53, 63,
       58, 24, 69, 35, 48, 33, 28, 65, 66, 13, 39, 55, 57, 53,  9])

In [21]:
# Decode these to see the text predicted by this untrained model:

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ir \nen una noche de amor \narrebatado por t\xc3\xad \nall\xc3\xa1 por Ituzaing\xc3\xb3. \n\ncamino donde la luna andaba de a '

Next Char Predictions:
 b'i\xc3\xb1s[UNK].  SVnR.qPdHzu y\xc3\xbaDkG\xc3\xb3xMsFD\xc3\xb1Ub\x0ct:Vcny\xc3\xbcjQ\xe2\x80\x99T\n\xc3\xbc:\xc3\xbcQ\xc3\x91beJofaeS\xe2\x80\xa6H q\xc3\xa9Hjmpzu![UNK]QODTYuzBxO\xe2\x80\xa6v\xc3\xb1\xc2\xb4P\xe2\x80\xa6dqbT\xc3\xba\xc3\xbcDhy\xc2\xa1v:'


# Train the model

In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


Prediction shape:  (32, 100, 70)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.24928


In [23]:
tf.exp(mean_loss).numpy()


70.054955

In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [26]:
EPOCHS = 50

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50
13/13 [==============================] - 2s 59ms/step - loss: 4.1244
Epoch 2/50
13/13 [==============================] - 1s 54ms/step - loss: 3.2484
Epoch 3/50
13/13 [==============================] - 1s 54ms/step - loss: 2.8767
Epoch 4/50
13/13 [==============================] - 1s 53ms/step - loss: 2.6755
Epoch 5/50
13/13 [==============================] - 1s 52ms/step - loss: 2.4786
Epoch 6/50
13/13 [==============================] - 1s 53ms/step - loss: 2.3530
Epoch 7/50
13/13 [==============================] - 1s 54ms/step - loss: 2.2708
Epoch 8/50
13/13 [==============================] - 1s 52ms/step - loss: 2.2054
Epoch 9/50
13/13 [==============================] - 1s 55ms/step - loss: 2.1642
Epoch 10/50
13/13 [==============================] - 1s 52ms/step - loss: 2.1256
Epoch 11/50
13/13 [==============================] - 1s 53ms/step - loss: 2.0953
Epoch 12/50
13/13 [==============================] - 1s 51ms/step - loss: 2.0691
Epoch 13/50
13/13 [==================

# Generate Text

In [30]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.5):
    super().__init__()  
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [31]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [32]:
start = time.time()
states = None
next_char = tf.constant(['rio'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


rio pos ajor  
casto de me encontrar. 

Fuinzó er la casta a cantar 
a llevar su canto mejor, 
en sus dulces since yabe ya tarde cuentas lejesarge lejos 
de la madrugada te despertaba 
Dorico te regresaba al anochecer cumpliera. 


Y a vos te lo diente siempre al pago donde nací, 
moriré por mal correntino 
en algún camino del taragüí llevó  
tu carita se forme, 
como un valiente también murió.   


Te dijo entonces sobre la muerte  
con cuando llegue de nuestra amor. 


Mi pasecí es mi vida de viajero triste 
que busca en vano adormecer su pena, 
por los cuerdos del mi monte te habrá de llamar, 
oniciera tus manos 
con alar de tara y día Solo este acuaban  
ti cosa gritante vieja en las palmeras  
para morirse acá hatá y naranjalera 
descalza en el arenal. 


Añoro tus quintas de enfilados naranjales 
tus chinas maduras quebradas sobre el agua 
y defiende su libertad. 


Partir, reir, bogar, soñar, 
un cigarrito en el andar, 
y piensa siempre mi correntina 
que aquí te espera mi coraz